#Data Cleanup
* find missing lats and longs
* find duplicates
* investigate unexpected lats and longs (some values appear not to be in Toronto)

In [1]:

from sqlalchemy.orm import sessionmaker
from models import connect_db, PointsOfInterest, ArchitecturalStyles, Architects,POICategories
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geopy.distance
import geocoder
import os
from dotenv import load_dotenv, find_dotenv
import time
DEBUG=0


In [2]:
import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)
logging.basicConfig(filename='../logs/datacleanup.log',level=logging.DEBUG)


In [3]:
def get_key(env='BING'):
    load_dotenv(find_dotenv())
    DEBUG = os.getenv("DEBUG")
    # load environment variables
    if env=='BING':
        KEY = os.getenv("BING_KEY")
    else:
        KEY = os.getenv("GOOGLE_KEY")
    #DEBUG = os.getenv("DEBUG")
    return KEY

In [20]:
def get_lats_longs(address):
    '''This function uses GeoCoder to retrieve the latitude and longitude for neighbourhoods.
    address to look up
    returns lat and long
    '''
    KEY = get_key("GOOGLE_KEY")
    # # get geocode for postal codes
    # initialize your variable to None
    lat_lng_coords = None
    lat=None
    long=None
    max_loops = 10
    curr_loop =0
    # loop until you get the coordinates (sometime dont' get a response)
    lat_lng_coords = None
    while(lat_lng_coords is None and curr_loop < max_loops):
      # g = geocoder.bing('{}'.format(address),key=KEY)
        g = geocoder.google('{}'.format(address), key=KEY)
       # g= geocoder.google('{}'.format(address))
        if DEBUG > 0:
            print (g)
        lat_lng_coords = g.latlng
        if DEBUG > 0:
            print(lat_lng_coords)
        if lat_lng_coords == None:
            time.sleep(1)
        curr_loop += 1
    if lat_lng_coords is not None:
        lat=lat_lng_coords[0]
        long=lat_lng_coords[1]

    return lat,long

In [5]:
def cleanup_address(row):
   
    lat=None
    long=None
    row.print_me()
    if a.address[0] == '0': 
        # look for "0 <street" format -- usually first digit
        print ("street address is zeroed")
        new_search = row.name + " " +  a.address[1:]  + ", Toronto ON"
        print(f'try searching on: {new_search}')
        lat, long=get_lats_longs(new_search)
    elif "Toronto" not in a.address:
         # check if missing Toronto and try adding it
        print('address is missing toronto')
        row.address = row.address +  ", Toronto ON"
        lat, long=get_lats_longs(row.address)
    print(f'new lat {lat} and long{long}')

    return lat, long

In [6]:
def update_missing_coords():
    '''
    Find all rows in PointsofINterest table that are missing lat and long
    Lookup address using Geocoder and update database row
    '''
    null_lats = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None)
#     row = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).first()
#     lat, long= get_lats_longs(row.address)
#     props = {'latitude':lat, 'longitude':long}

#     for key, value in props.items():
#         setattr(row, key, value)

    for row in null_lats:
        #null_lat = session.query(PointsOfInterest).get(1)
        lat, long= get_lats_longs(row.address)
        props = {'latitude':lat, 'longitude':long}

        for key, value in props.items():
            setattr(row, key, value)

    # save up our commits
    session.commit()
    session.flush()
    return True


In [7]:
def check_missing_coords():
    num_missing_coords = session.query(PointsOfInterest).filter(PointsOfInterest.latitude == None).count()
    print(f"There are {num_missing_coords} records missing lat/long coordindates.")
    if num_missing_coords> 0:
        update_missing_coords()

# Clean up duplicate rows

In [ ]:
def delete_poi_by_id(id):
    poi = session.query(PointsOfInterest).filter(PointsOfInterest.poi_id==id).first()
    print(f"deleting {poi.name}")
    session.delete(poi)
    session.commit()
    return True

In [14]:
def find_and_delete_dups(df):
    df.sort_values('name', inplace=True)
    prev_name=''
    prev_addr = ''
    prev_src =  ''
    prv_exturl=''
    for ix, row in df.iterrows():
        if prev_name == row['name'] and prev_addr == row['address'] and  prev_src==row['source'] and prv_exturl==row['external_url']:
            print(f"duplicate {prev_name} at {prev_addr} from {prev_src} and {prv_exturl}")
            delete_poi_by_id(row['poi_id'])
            #print(row)
        prev_name = row['name']
        prev_addr = row['address']
        prev_src = row['source']
        prv_exturl=row['external_url']

# Try to find points outside Toronto

In [35]:
def do_address_fix(address):
    if 'ON' not in address:
        address = f"{address}, ON"
    return address

In [36]:
def update_coords(id, fix_address=False):
    poi_to_update = session.query(PointsOfInterest).filter(PointsOfInterest.poi_id == id).first()
    print(f"Checking coordinates for{poi_to_update.name}. Current coordinates for {poi_to_update.address} are {poi_to_update.latitude}, {poi_to_update.longitude}")
    logging.debug(f"Checking coordinates for {id} {poi_to_update.name}. Current coordinates for {poi_to_update.address} are {poi_to_update.latitude}, {poi_to_update.longitude}")
    
    if fix_address:
        poi_to_update.address=do_address_fix(poi_to_update.address)
        print(f'new address{poi_to_update.address}')
    
    lat, long = get_lats_longs(poi_to_update.address)
    logging.debug(f"Updating to {lat},{long}")
    
    poi_to_update.latitude = lat
    poi_to_update.longitude =long

    session.commit()
    session.flush()

    print(lat, long)

In [37]:
def find_dist(df, lat, long):
    avail_points = []
    
    for ix, row in df.iterrows():
        curr_pt = geopy.distance.geodesic((row['latitude'], row['longitude']), (lat, long)).km
        #print(curr_pt)
        
        avail_points.append(curr_pt)
    df['dist_start'] = avail_points
    return df

In [38]:
def find_points_outside_TO(df, fix_address=False, dist=50, starting_lat=43.656287,starting_long= -79.380898):
    ''' Find points more than 50KM from downtown Toronto (Yonge Dundas Square is default) and try to update'''
    df=find_dist(df, starting_lat, starting_long)
    for ix, row in df[df['dist_start']>dist].iterrows():
        print(f"{row['poi_id'], row['name']} is outside of Toronto")
        update_coords(row['poi_id'], fix_address)

In [40]:
db=connect_db() #establish connection / creates database on first run
Session = sessionmaker(bind=db)
session = Session()

#check_missing_coords()

# remove duplicate rows
# df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
# df.sort_values('name', inplace=True)
# find_and_delete_dups(df)

# get fresh df from database with removed deletes
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.sort_values('poi_id', inplace=True)
find_points_outside_TO(df)

# try again in case still have bad addresses -- but this time try to fix them
df = pd.read_sql(session.query(PointsOfInterest).statement, session.bind)
df.sort_values('poi_id', inplace=True)
find_points_outside_TO(df,fix_address=True)

(8, '43 Cross Street') is outside of Toronto
Checking coordinates for43 Cross Street. Current coordinates for 43 Cross Street Weston York are 39.9625984, -76.727745
39.9625984 -76.727745
(61, '6 Fern Avenue') is outside of Toronto
Checking coordinates for6 Fern Avenue. Current coordinates for 6 Fern Avenue Weston York are 43.2994285, -74.2179326
43.2994285 -74.21793260000001
(3245, 'William Muir House (Howard Street)') is outside of Toronto
Checking coordinates forWilliam Muir House (Howard Street). Current coordinates for 8 Howard Street St. Jamestown East York are 42.0838943, -79.2340753
42.0838943 -79.23407530000001
(8, '43 Cross Street') is outside of Toronto
Checking coordinates for43 Cross Street. Current coordinates for 43 Cross Street Weston York are 39.9625984, -76.727745
new address43 Cross Street Weston York, ON
43.7050882 -79.5255011
(61, '6 Fern Avenue') is outside of Toronto
Checking coordinates for6 Fern Avenue. Current coordinates for 6 Fern Avenue Weston York are 43.29